In [9]:
%matplotlib inline
import pandas as pd
import matplotlib as mat

# Carregamento e limpeza dos dados

In [73]:
data = pd.read_csv("data_group5_fixed.csv")

#Retirando inconsistencias na media
data["Media"] = data["Media"].apply(lambda x:x[0:3] if isinstance(x, basestring) else x)


data["Cidade"] = data["Cidade"].apply(lambda x:x[:-1] if x and x[-1].isspace() else x)
data["Cidade"] = data["Cidade"].apply(lambda x:x[x.find("Novo Hamburgo"):13] if x.find("Novo Hamburgo") != -1 else x)
data["Cidade"] = data["Cidade"].apply(lambda x:"Canguçu" if x == "Cangucu" else x)

#print(data.loc[[20]])
# data[data['Cidade'] == "Canguçu"]

                    Curso  Semestre_ingresso  Idade         Cidade Estado  \
20  Ciência da Computação                172     21  Capão do Leão     RS   

   Trabalha             Forma_ingresso Cota  Num_disciplinas  Reprovacao Media  
20      Sim  SISU - Ampla concorrência  NaN               16           0   9.1  


In [11]:
data.shape

(176, 11)

In [12]:
dist = pd.read_csv("distancias_de_pelotas.csv")
dist.head()

,cidade,latitude,longitude,distancia
0,Abadia Dos Dourados,-18.491063,-47.406365,1552.553136
1,Abaetetuba,-1.718060,-48.882500,3346.389498
2,Abaiara,-7.336420,-39.061290,3036.331976
3,Abatia,-23.305297,-50.310253,959.381035
4,Abdon Batista,-27.589810,-51.040230,480.395004


# DISTANCIA DE CASA AFETA A MEDIA?

In [83]:
filtered = data[["Media", "Num_disciplinas", "Reprovacao", "Cidade"]]
#city = filtered.groupby('Cidade')
#city.head()
city = filtered.groupby('Cidade').count()
city

#filtered['Cidade'].sort_values()

#filtered.groupby('Cidade').count().sort_values('Cidade',ascending=False)

,Media,Num_disciplinas,Reprovacao
Cidade,,,
Alegrete,1,1,1
Alvorada,1,1,1
Bacabal,1,1,1
Bagé,2,2,2
Barbacena,1,1,1
Belém,1,1,1
Betim,1,1,1
Botucatu,1,1,1
Camaquã,5,5,5


#Tirando Cidade == Rio Grande do sul

In [85]:
city.drop(['Rio Grande do Sul'])

,Media,Num_disciplinas,Reprovacao
Cidade,,,
Alegrete,1,1,1
Alvorada,1,1,1
Bacabal,1,1,1
Bagé,2,2,2
Barbacena,1,1,1
Belém,1,1,1
Betim,1,1,1
Botucatu,1,1,1
Camaquã,5,5,5
